In [1]:
import pandas as pd
import torch
import sklearn
import numpy as np

from transformers import AutoTokenizer, AutoConfig, EarlyStoppingCallback, AutoModelForSequenceClassification, Trainer, TrainingArguments, RobertaConfig, RobertaTokenizer, RobertaForSequenceClassification, BertTokenizer

In [13]:
tokenizer = AutoTokenizer.from_pretrained("goodjw/klue-roberta-large-tapt")
tokenizer.decode(tokenizer.encode("〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다"))

Downloading:   0%|          | 0.00/337 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/752 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

'[CLS] 〈 Something 〉 는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《 Abbey Road 》 에 담은 노래다 [SEP]'

In [14]:
tokenizer.tokenize("〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다")

['〈',
 'So',
 '##me',
 '##th',
 '##ing',
 '〉',
 '는',
 '조지',
 '해리',
 '##슨',
 '##이',
 '쓰',
 '##고',
 '비틀즈',
 '##가',
 '1969',
 '##년',
 '앨범',
 '《',
 'Ab',
 '##be',
 '##y',
 'Ro',
 '##ad',
 '》',
 '에',
 '담',
 '##은',
 '노래',
 '##다']

In [15]:
df = pd.read_csv("../dataset/train/train.csv")
df

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia
...,...,...,...,...,...,...
32465,32465,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,"{'word': '유기준', 'start_idx': 93, 'end_idx': 95...","{'word': '부산 서구·동구', 'start_idx': 100, 'end_id...",per:employee_of,wikitree
32466,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...",per:colleagues,wikipedia
32467,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...","{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...",org:top_members/employees,wikitree
32468,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...",no_relation,wikipedia


In [ ]:
import os
from pathlib import Path
df = pd.read_csv("../dataset/train/train.csv")

def read_file(file_list):
    token_docs = []
    tag_docs = []
    for i in df.index:
        
        sentence = df['sentence'].iloc[i]
        for j,k in zip(df['subject_entity'], df['object_entity']):
            subject_start = int(i[1:-1].split(',')[1].split(':')[1])
            subject_end = int(i[1:-1].split(',')[2].split(':')[1])
            subject_type = i[1:-1].split(',')[3].split(':')[1][2:-1]
            object_start = int(k[1:-1].split(',')[1].split(':')[1])
            object_end = int(k[1:-1].split(',')[2].split(':')[1])
            object_type = k[1:-1].split(',')[3].split(':')[1][2:-1]
        
        for idx in sentence:
            tokens = []
            tags = []
            if idx == subject_start:
                

            token_docs.append(tokens)
            tag_docs.append(tags)

    return token_docs, tag_docs

In [23]:
print(df['subject_entity'].iloc[0][1:-1].split(','))
int(df['subject_entity'].iloc[0][1:-1].split(',')[1].split(':')[1])
int(df['subject_entity'].iloc[0][1:-1].split(',')[2].split(':')[1])

["'word': '비틀즈'", " 'start_idx': 24", " 'end_idx': 26", " 'type': 'ORG'"]


26

In [32]:
print(df['object_entity'].iloc[0][1:-1].split(','))
df['object_entity'].iloc[0][1:-1].split(',')[1].split(':')[1]
df['object_entity'].iloc[0][1:-1].split(',')[2].split(':')[1]
df['object_entity'].iloc[0][1:-1].split(',')[3].split(':')[1][2:-1]

["'word': '조지 해리슨'", " 'start_idx': 13", " 'end_idx': 18", " 'type': 'PER'"]


'PER'

In [42]:
print(df[df['label'] == "per:colleagues"])

          id                                           sentence  \
20        20  1971년 대선을 앞두고 김종필은 1971년 선거에서 박정희 당선을 위해 무려 60...   
157      157  아나킨 스카이워커는 미디클로리언 수치가 20000이 넘는, 선천적으로 가진 포스의 ...   
276      276  《Riding with the King》은 2000년 발매된 에릭 클랩튼과 B.B....   
282      282  2017년 문재인 정부의 정현백 장관은 "여성혐오에 소극적으로 대처하는 건 더이상 ...   
321      321  요한 바오로 2세는 요제프 라칭거 추기경(훗날의 교황 베네딕토 16세)과 함께 자유...   
...      ...                                                ...   
32188  32188  그 결과로서 밴스는 존 F. 케네디와 린든 B. 존슨 행정부에서 직위들의 계승에 근...   
32277  32277  한편 김대중은 김종필, 박태준과 단일화에 합의하였고, 이에 불만을 품은 신한국당 역...   
32284  32284  초반에는 도쿄에서 한 마을인 히나미자와 마을에 전학 온지 얼마 안된 소년 마에바라 ...   
32328  32328  문 대통령은 이어진 환담 자리에서 싱 대사에게 지난 해 방중 시 시진핑 주석과 리커...   
32466  32466  법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...   

                                          subject_entity  \
20     {'word': '김종필', 'start_idx': 14, 'end_idx': 16...   
157    {'word': '아나킨 스카이워커', 'start_idx': 0, 'end_idx...   
276    {'word':

In [46]:
print(df.iloc[32284]['sentence'])
print(df.iloc[32284]['subject_entity'])
print(df.iloc[32284]['object_entity'])
print(df.iloc[32284]['label'])

초반에는 도쿄에서 한 마을인 히나미자와 마을에 전학 온지 얼마 안된 소년 마에바라 케이이치와 개성적인 동아리의 동료들과의 일상을 그리지만 중반 이후에 갑자기 발생하는 괴사건과 서스펜스가 공포 분위기를 두드러지게 한다.
{'word': '마에바라 케이이치', 'start_idx': 41, 'end_idx': 49, 'type': 'PER'}
{'word': '히나미자와', 'start_idx': 16, 'end_idx': 20, 'type': 'LOC'}
per:colleagues


In [50]:
for i in df.index:
    if df['label'].iloc[i] == "per:colleagues":
        if df.iloc[i]['subject_entity'][1:-1].split(',')[3].split(':')[1].strip() == "'ORG'" or df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1].strip() == "'ORG'":
            print(df.iloc[i]['sentence'], i)

서병수 전 부산시장이 유시민 노무현재단 이사장을 향해 쓴소리했다. 2321
29일 TBS '김어준의 뉴스공장'에 유시민 노무현재단 이사장이 출연했다. 3358
유시민 노무현재단 이사장이 백년이 지나도 한국의 언론개혁이 이루어 지지 않을 것으로 내다봤다. 4430
강인은 "무엇보다 14년이란 오랜 시간 동안 언제나 과분한 사랑을 주신 E.L.F. 여러분들께 가장 죄송한 마음입니다"라며 "많이 늦었지만 슈퍼주니어란 이름을 내려놓고 홀로 걷는 길에도 항상 미안함과 고마움을 가슴에 새기고 나아가겠습니다"라고 했다. 4534
8월 29일, 유시민 노무현재단 이사장은 서울대학교 촛불 집회에 대하여 "뒤에서 자유한국당 손길이 어른어른하는 거라고 본다"며, 왜 마스크를 쓰고 집회하냐고 발언하여 논란을 빚었다. 7754
그다음 시즌, 카이저슬라우테른은 1947 시즌에 프리츠 발터와 오트마어 발터에 큰힘을 입어 북부 조 (Gruppe Nord) 에서 손쉽게 우승을 거두었다. 13812
유시민 노무현재단 이사장이 조국 사태에 "인간이 무섭다는 생각이 든다"고 말했다. 17550
와타나베는 원래 [[연병관]](練兵館)의 우두머리로, 조슈 번의 중신 [[기도 다카요시]]나 [[다카스기 신사쿠]] 등과 막역한 사이였기 때문에, 료마는 와타나베에게 조슈 번에 로비를 의뢰했다. 20409
유시민 노무현재단 이사장이 KBS 법조팀과 검찰의 유착 의혹을 제기했다. 22518
산투스 복귀 후, 호비뉴는 발뒤꿈치로 그의 영입을 갈망했던 상파울루를 상대로 경기 종료 5분을 남기고 득점했고, "물고기 군단"은 2-1 승리를 쟁취했다. 25367


In [51]:
print(df.iloc[2321]['sentence'])
print(df.iloc[2321]['subject_entity'])
print(df.iloc[2321]['object_entity'])
print(df.iloc[2321]['label'])

서병수 전 부산시장이 유시민 노무현재단 이사장을 향해 쓴소리했다.
{'word': '유시민', 'start_idx': 12, 'end_idx': 14, 'type': 'PER'}
{'word': '노무현', 'start_idx': 16, 'end_idx': 18, 'type': 'ORG'}
per:colleagues


사람과 사람 관계이어야 하는 것들 확인

In [70]:
k=0
for i in df.index:
    if df['label'].iloc[i] == "per:colleagues" or df['label'].iloc[i] == "per:siblings" or df['label'].iloc[i] == "per:children" or df['label'].iloc[i] == "per:parents" or df['label'].iloc[i] == "per:spouse" or df['label'].iloc[i] == "per:other_family" or df['label'].iloc[i] == "per:other_family":
        if (df.iloc[i]['subject_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'PER'" and df.iloc[i]['subject_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'POH'") or (df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'PER'" and df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'POH'"):
            print(df.iloc[i]['sentence'], i)
            k+=1
print(k)

1725년(영조 1) 2월 18일에 화순옹주로 봉해졌으며, 1732년(영조 8) 영의정을 지낸 김흥경(金興慶)의 아들이자 추사 김정희의 증조부인 월성위(月城尉) 김한신(金漢藎)과 혼례를 치렀다. 708
양평군의 어머니 숙의 이씨는 양성 이씨 출신으로, 중종 때 판의금부사와 좌찬성 등을 역임한 이손과 일가 친척이다. 843
아버지는 덴마크 왕 덴마크의 크리스티안 9세, 어머니는 헤센카셀의 루이세이다. 1504
그의 아들 성삼문, 성삼빙, 성삼고, 성삼성과 손자 네 명이 모두 처형당하여 그의 후사는 끊겼다. 가까운 친척으로는 그의 사촌 성희의 후손들이 살아남아 현존하고 있다. 1729
로이히텐베르크의 요세피나는 조제핀이라는 이름으로 외젠 드 보아르네와 바이에른 공주 아우구스테의 장녀로 태어났다. 1992
서병수 전 부산시장이 유시민 노무현재단 이사장을 향해 쓴소리했다. 2321
29일 TBS '김어준의 뉴스공장'에 유시민 노무현재단 이사장이 출연했다. 3358
공빈 김씨(恭嬪 金氏, 1553년 11월 16일 ~ 1577년 6월 13일) 또는 공성왕후(恭聖王后)는 조선의 제 14대 임금 선조의 빈이자 추존왕후이며 임해군, 광해군 형제의 생모이다. 3382
마우리아-셀레우코스 전쟁 이후 찬드라굽타 마우리아는 인도 아대륙 안쪽으로 관심을 돌려 북인도뿐만 아니라 중부 인도와 남인도 지역에도 확장을 시도하였고 그 결과 마우리아 제국은 오디샤 지역의 칼링가와 타밀라캄 지역의 촐라, 판디아, 체라를 제외한 인도 아대륙 대부분을 통일하였지만, 찬드라굽타 마우리아 재위 말기인 기원전 298년에 마우리아 제국의 영토에 발생한 극심한 가뭄으로 인해 수많은 사람들이 기아로 사망하는 일이 발생하자 이에 충격을 받은 찬드라굽타 마우리아는 당시 그가 스승으로 모시던 자이나교 구루의 충고를 들은 후 정치에 손을 떼고 아들인 빈두사라에게 마우리아 제국의 황제 자리를 물려주었으며, 자이나교에 귀의하여 출가한 후 고행을 하다가 일생을 마쳤다. 4077
남연군 이구와 여흥군부인 여흥 민씨의 셋째 아

In [69]:
print(df.iloc[32320]['sentence'])
print(df.iloc[32320]['subject_entity'])
print(df.iloc[32320]['object_entity'])
print(df.iloc[32320]['label'])

칭기즈 칸의 씨족인 보르지긴은 황금씨족(알탄 우룩)이라 불리며, 영지 백성(우르스)으로 나뉜 천호・백호・십호 집단의 위에 상급 영주 계급으로써 군림했고, 몽골 황제 즉 대칸은 크고 작은 우르스의 가장 큰 부분을 가진 맹주였다.
{'word': '칭기즈 칸', 'start_idx': 0, 'end_idx': 4, 'type': 'PER'}
{'word': '보르지긴', 'start_idx': 11, 'end_idx': 14, 'type': 'ORG'}
per:other_family


In [72]:
k=0
for i in df.index:
    if df['label'].iloc[i] == "per:date_of_birth" or df['label'].iloc[i] == "per:date_of_death":
        if (df.iloc[i]['subject_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'PER'" and df.iloc[i]['subject_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'POH'") or (df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'DAT'"):
            print(df.iloc[i]['sentence'], i)
            k+=1
print(k)

발레리나 출신인 최혜영씨는 1979년생으로 무용학과를 졸업했지만 2003년 교통사고로 사지마비 척수장애를 입고 무용수의 꿈을 접고, 여성 척수장애인 국내 최초로 재활학 박사가 됐다. 3033
동치제(同治帝, 함풍(咸豊)(咸豊) 6년 3월 23일(1856년 4월 27일) ~ 동치(同治) 13년 12월 5일(1875년 1월 12일))은 중국 청 왕조의 제10대 황제(재위 1861년 ~ 1874년)이다. 6671
광서제(光緖帝, 동치(同治)(同治) 10년 6월 28일(1871년 8월 14일) ~ 광서(光緖) 34년 10월 21일(1908년 11월 14일))는 청나라의 제11대 황제이다. 14741
홍희제(洪熙帝, 홍무(洪武)(洪武) 11년 7월 23일(1378년 8월 16일) ~ 홍희(洪熙) 원년 5월 12일(1425년 5월 29일))는 중국 명나라의 제4대 황제(재위 1424년~1425년)이다. 18279
페르시아 제국의 다리우스 3세가 알렉산드로스 대왕에게 패하자 그를 살해한 박트리아의 총독 베수스는 민족적 저항을 조직하려고 하였다. 19531
우키타 히데이에(겐키 3년(1572년)(1572년) ~ 메이레키 원년(1655년 12월 17일))는 아즈치모모야마 시대의 비젠 오카야마성 다이묘이다. 20742
브라마굽타(597–668)는 펠 방정식의 연구를 시작하였으며, 자야데바(Jayadeva, 9세기 경)가 펠 방정식의 해를 완성하였다. 22631
쩐 영종(바오푸(寶符)(寶符) 4년 9월 17일(1276년 10월 27일) ~ 다이카인(大慶) 7년 3월 16일(1320년 4월 24일))은 베트남 쩐 왕조의 제4대 황제(재위 : 1293년 ~ 1314년)이다. 27484
하지만, 김춘추와 문희의 첫번째 아들인 김법민(문무왕)이 626년생으로, 문희가 김법민을 임신했을때는 선덕여왕이 즉위하기 5년전인 진평왕 때라는 것이 된다. 27954
요기 베라는 자신의 메이저 데뷔 일과 같은 날로 내보였던 2015년 9월 22일에는 뉴저지주 웨스트 칼드웰에 있는 고령자 집합 주택에서

In [74]:
print(df.iloc[3033]['sentence'])
print(df.iloc[3033]['subject_entity'])
print(df.iloc[3033]['object_entity'])
print(df.iloc[3033]['label'])

발레리나 출신인 최혜영씨는 1979년생으로 무용학과를 졸업했지만 2003년 교통사고로 사지마비 척수장애를 입고 무용수의 꿈을 접고, 여성 척수장애인 국내 최초로 재활학 박사가 됐다.
{'word': '최혜영', 'start_idx': 9, 'end_idx': 11, 'type': 'PER'}
{'word': '1979년', 'start_idx': 15, 'end_idx': 19, 'type': 'NOH'}
per:date_of_birth


In [75]:
print(df.iloc[6671]['sentence'])
print(df.iloc[6671]['subject_entity'])
print(df.iloc[6671]['object_entity'])
print(df.iloc[6671]['label'])

동치제(同治帝, 함풍(咸豊)(咸豊) 6년 3월 23일(1856년 4월 27일) ~ 동치(同治) 13년 12월 5일(1875년 1월 12일))은 중국 청 왕조의 제10대 황제(재위 1861년 ~ 1874년)이다.
{'word': '동치제', 'start_idx': 0, 'end_idx': 2, 'type': 'PER'}
{'word': '동치(同治) 13년 12월 5일(1875년 1월 12일)', 'start_idx': 46, 'end_idx': 76, 'type': 'PER'}
per:date_of_death
